# Layer 6 False Alarm Reduction Demo

This notebook demonstrates:
- Layer 6 peer-consistency and regime tagging flow,
- heuristic triage action generation,
- optional supervised triage mode with demo labels,
- output audit artifact inspection.

In [ ]:
from pathlib import Path
import json
import subprocess
import pandas as pd

PROJECT_ROOT = Path.cwd()
PROJECT_ROOT

## Run Layer 6 Demo (Heuristic)

In [ ]:
cmd = [
    "python", "-m", "pipeline.run_demo_layer6_false_alarm",
    "--universe-name", "CP_RATE_CORP",
    "--start-date", "2024-01-01",
    "--end-date", "2024-03-31",
    "--business-date", "2024-03-29",
    "--num-factors", "150",
    "--max-series", "20",
    "--output-format", "parquet"
]
result = subprocess.run(cmd, cwd=PROJECT_ROOT, capture_output=True, text=True, check=True)
print(result.stdout)

## Locate Latest Output

In [ ]:
root_base = PROJECT_ROOT / "data" / "processed" / "layer6_false_alarm_demo" / "business_date=2024-03-29" / "universe_name=CP_RATE_CORP"
run_dirs = sorted([p for p in root_base.iterdir() if p.is_dir() and p.name.startswith("run_id=")])
latest_root = run_dirs[-1]
latest_root

## Inspect Manifest

In [ ]:
manifest = json.loads((latest_root / "manifest.json").read_text(encoding="utf-8"))
manifest

## Load Main Layer 6 Outputs

In [ ]:
triaged_preview = pd.read_parquet(latest_root / "triaged_preview.parquet")
summary_by_action = pd.read_parquet(latest_root / "summary_by_action.parquet")
summary_by_regime = pd.read_parquet(latest_root / "summary_by_regime.parquet")
summary_by_check = pd.read_parquet(latest_root / "summary_by_check.parquet")
peer_summary = pd.read_parquet(latest_root / "peer_summary.parquet")
triage_model = pd.read_parquet(latest_root / "triage_model.parquet")

print("triaged_preview:", triaged_preview.shape)
print("summary_by_action:", summary_by_action.shape)
print("summary_by_regime:", summary_by_regime.shape)
print("summary_by_check:", summary_by_check.shape)
print("peer_summary:", peer_summary.shape)
print("triage_model:", triage_model.shape)
summary_by_action

## Optional: Supervised Mode
Re-run the command with `--with-supervised --generate-demo-labels --label-rate 0.15` to populate model metrics and probabilities.